# Module 2.1: Otsu's Method（大津法）

## 學習目標

本 notebook 將帶你實作 **Otsu's Method**：

1. 理解二值化分割的問題定義
2. 推導類間變異數（Between-Class Variance）
3. 從零實作 Otsu 自動閾值選擇
4. 理解為什麼最大化類間變異等於最小化類內變異

## 前置要求
- 完成 Module 0（機率基礎）
- 完成 Module 1.3（直方圖）

## 參考資料
- Otsu, 1979. "A Threshold Selection Method from Gray-Level Histograms"
- Gonzalez & Woods, Ch. 10.3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys
sys.path.append('../../')

try:
    from utils.image_io import load_grayscale, save_image
except:
    def load_grayscale(path):
        return np.array(Image.open(path).convert('L'), dtype=np.float64)
    def save_image(arr, path):
        Image.fromarray(arr.astype(np.uint8)).save(path)

print("環境設定完成！")

## 建立測試影像

建立一些適合二值化分割的測試影像。

In [ ]:
def create_bimodal_image(size=200):
    """
    建立一個雙峰分布的影像（理想的二值化案例）。
    前景和背景有明顯的灰階差異。
    """
    np.random.seed(42)
    image = np.zeros((size, size), dtype=np.float64)
    
    # 背景：暗色（均值 50）
    image[:] = 50 + np.random.randn(size, size) * 15
    
    # 前景物件：亮色（均值 180）
    # 物件 1：大圓形
    y, x = np.ogrid[:size, :size]
    center_y, center_x = size//3, size//3
    radius = size//5
    mask1 = (x - center_x)**2 + (y - center_y)**2 <= radius**2
    image[mask1] = 180 + np.random.randn(*image[mask1].shape) * 15
    
    # 物件 2：方塊
    image[size//2:size//2+size//4, size//2:size//2+size//4] = (
        180 + np.random.randn(size//4, size//4) * 15
    )
    
    # 物件 3：橢圓
    center_y, center_x = 2*size//3, size//4
    mask2 = ((x - center_x)/30)**2 + ((y - center_y)/15)**2 <= 1
    image[mask2] = 180 + np.random.randn(*image[mask2].shape) * 15
    
    return np.clip(image, 0, 255).astype(np.uint8)


def create_gradient_image(size=200):
    """
    建立一個漸層影像（較難二值化）。
    """
    gradient = np.linspace(0, 255, size)
    image = np.tile(gradient, (size, 1))
    noise = np.random.randn(size, size) * 10
    return np.clip(image + noise, 0, 255).astype(np.uint8)


def create_text_like_image(size=200):
    """
    建立類似文字/印刷品的影像。
    """
    np.random.seed(42)
    image = np.ones((size, size), dtype=np.float64) * 220  # 白色背景
    
    # 加入「文字」（黑色線條）
    for i in range(5):
        y_start = 20 + i * 35
        # 水平線
        image[y_start:y_start+8, 20:180] = 30
        # 加入一些「字元」（方塊）
        for j in range(8):
            x_start = 20 + j * 20
            if np.random.rand() > 0.3:
                image[y_start:y_start+15, x_start:x_start+15] = 30
    
    # 加入雜訊
    noise = np.random.randn(size, size) * 10
    return np.clip(image + noise, 0, 255).astype(np.uint8)


# 建立測試影像
bimodal_img = create_bimodal_image()
gradient_img = create_gradient_image()
text_img = create_text_like_image()

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(bimodal_img, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Bimodal (Ideal Case)')

axes[1].imshow(gradient_img, cmap='gray', vmin=0, vmax=255)
axes[1].set_title('Gradient (Challenging)')

axes[2].imshow(text_img, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('Text-like')

for ax in axes:
    ax.axis('off')

plt.tight_layout()
plt.show()

---

# Part 1: 問題定義

## 二值化分割 (Binary Thresholding)

**目標**：找一個閾值 $T$，將影像像素分成兩類：

$$
g(x, y) = 
\begin{cases}
1 & \text{if } f(x, y) > T \\
0 & \text{otherwise}
\end{cases}
$$

### 什麼是「好的」閾值？

直覺上，好的閾值應該：
- 讓同一類內的像素值相近（**類內變異小**）
- 讓不同類別的像素值差異大（**類間變異大**）

### Otsu 的洞見

最大化**類間變異數 (Between-Class Variance)** 等於最小化**類內變異數 (Within-Class Variance)**

In [ ]:
# 先觀察直方圖
def compute_histogram(image, bins=256):
    """計算直方圖（複習 Module 1.3）"""
    flat = image.flatten().astype(np.int32)
    flat = np.clip(flat, 0, bins - 1)
    return np.bincount(flat, minlength=bins)[:bins]

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

images = [bimodal_img, gradient_img, text_img]
titles = ['Bimodal', 'Gradient', 'Text-like']

for i, (img, title) in enumerate(zip(images, titles)):
    # 影像
    axes[0, i].imshow(img, cmap='gray', vmin=0, vmax=255)
    axes[0, i].set_title(f'{title} Image')
    axes[0, i].axis('off')
    
    # 直方圖
    hist = compute_histogram(img)
    axes[1, i].bar(range(256), hist, width=1, color='gray', edgecolor='none')
    axes[1, i].set_xlim(0, 255)
    axes[1, i].set_title(f'{title} Histogram')
    axes[1, i].set_xlabel('Pixel Intensity')
    axes[1, i].set_ylabel('Count')

plt.tight_layout()
plt.show()

print("觀察：")
print("1. Bimodal：直方圖有兩個明顯的峰（雙峰分布），理想的二值化案例")
print("2. Gradient：直方圖均勻分布，沒有自然的分界")
print("3. Text-like：也是雙峰，但峰值比例不同")

---

# Part 2: 數學推導

## 符號定義

設影像有 $L$ 個灰階（0 到 $L-1$），總像素數為 $N$。

- **直方圖**：$h(k)$ = 灰階 $k$ 的像素數量
- **正規化直方圖（機率）**：$p(k) = h(k) / N$

對於閾值 $T$，將像素分成兩類：
- **Class 0 (Background)**：灰階 $\leq T$
- **Class 1 (Foreground)**：灰階 $> T$

## 各類別的統計量

### 類別權重（累積機率）

$$
\omega_0(T) = \sum_{k=0}^{T} p(k) = P(\text{pixel} \leq T)
$$

$$
\omega_1(T) = \sum_{k=T+1}^{L-1} p(k) = 1 - \omega_0(T)
$$

### 類別平均值

$$
\mu_0(T) = \frac{\sum_{k=0}^{T} k \cdot p(k)}{\omega_0(T)}
$$

$$
\mu_1(T) = \frac{\sum_{k=T+1}^{L-1} k \cdot p(k)}{\omega_1(T)}
$$

### 全域平均值

$$
\mu_T = \sum_{k=0}^{L-1} k \cdot p(k) = \omega_0 \mu_0 + \omega_1 \mu_1
$$

## 類間變異數 (Between-Class Variance)

$$
\sigma_B^2(T) = \omega_0(T) \cdot (\mu_0(T) - \mu_T)^2 + \omega_1(T) \cdot (\mu_1(T) - \mu_T)^2
$$

可以簡化為：

$$
\sigma_B^2(T) = \omega_0(T) \cdot \omega_1(T) \cdot (\mu_0(T) - \mu_1(T))^2
$$

### Otsu 的目標：找使 $\sigma_B^2(T)$ 最大的 $T$

In [ ]:
# 用一個簡單的例子驗證公式
def demonstrate_otsu_math():
    """
    用一個簡單的 10x10 影像示範 Otsu 的數學。
    """
    # 建立一個簡單的影像：左半暗（50），右半亮（200）
    simple_img = np.zeros((10, 10), dtype=np.uint8)
    simple_img[:, :5] = 50
    simple_img[:, 5:] = 200
    
    # 計算直方圖
    hist = compute_histogram(simple_img)
    total_pixels = simple_img.size
    p = hist / total_pixels  # 正規化直方圖
    
    print("=== 簡單範例 ===")
    print(f"影像：左半是 50，右半是 200")
    print(f"總像素數：{total_pixels}")
    print(f"灰階 50 的像素數：{hist[50]}")
    print(f"灰階 200 的像素數：{hist[200]}")
    print()
    
    # 測試閾值 T = 100（在兩個灰階值之間）
    T = 100
    
    # 計算 omega_0 和 omega_1
    omega_0 = p[:T+1].sum()  # 灰階 <= T 的機率
    omega_1 = p[T+1:].sum()  # 灰階 > T 的機率
    
    print(f"閾值 T = {T}：")
    print(f"  ω₀ = {omega_0:.3f} (Class 0 的權重)")
    print(f"  ω₁ = {omega_1:.3f} (Class 1 的權重)")
    print(f"  ω₀ + ω₁ = {omega_0 + omega_1:.3f}")
    print()
    
    # 計算 mu_0 和 mu_1
    k = np.arange(256)
    
    # 避免除以零
    if omega_0 > 0:
        mu_0 = (k[:T+1] * p[:T+1]).sum() / omega_0
    else:
        mu_0 = 0
    
    if omega_1 > 0:
        mu_1 = (k[T+1:] * p[T+1:]).sum() / omega_1
    else:
        mu_1 = 0
    
    mu_T = (k * p).sum()  # 全域平均
    
    print(f"  μ₀ = {mu_0:.3f} (Class 0 的平均灰階)")
    print(f"  μ₁ = {mu_1:.3f} (Class 1 的平均灰階)")
    print(f"  μ_T = {mu_T:.3f} (全域平均)")
    print(f"  驗證：ω₀μ₀ + ω₁μ₁ = {omega_0*mu_0 + omega_1*mu_1:.3f}")
    print()
    
    # 計算類間變異數
    sigma_B_sq = omega_0 * omega_1 * (mu_0 - mu_1)**2
    
    print(f"  σ_B² = ω₀ · ω₁ · (μ₀ - μ₁)²")
    print(f"       = {omega_0:.3f} × {omega_1:.3f} × ({mu_0:.3f} - {mu_1:.3f})²")
    print(f"       = {sigma_B_sq:.3f}")
    
    return simple_img

simple_img = demonstrate_otsu_math()

---

# Part 3: 實作 Otsu's Method

## 演算法步驟

1. 計算影像的直方圖
2. 對每個可能的閾值 $T$ (0 到 255)：
   - 計算 $\omega_0(T)$, $\omega_1(T)$
   - 計算 $\mu_0(T)$, $\mu_1(T)$
   - 計算 $\sigma_B^2(T)$
3. 找到使 $\sigma_B^2(T)$ 最大的 $T$

In [ ]:
def otsu_threshold_naive(image):
    """
    Naive 實作：遍歷所有可能的閾值。
    
    Parameters
    ----------
    image : np.ndarray
        灰階影像 (uint8)
    
    Returns
    -------
    threshold : int
        最佳閾值
    sigma_B_sq : np.ndarray
        每個閾值對應的類間變異數（用於視覺化）
    
    實作步驟：
    1. 計算直方圖並正規化
    2. 對每個 T 從 0 到 255：
       - 計算 omega_0, omega_1
       - 計算 mu_0, mu_1
       - 計算 sigma_B_sq
    3. 回傳最大 sigma_B_sq 對應的 T
    """
    # Step 1: 計算直方圖
    hist = compute_histogram(image, bins=256)
    total_pixels = image.size
    p = hist.astype(np.float64) / total_pixels  # 正規化
    
    # 灰階值
    k = np.arange(256)
    
    # 全域平均
    mu_total = np.sum(k * p)
    
    # 儲存每個 T 的 sigma_B_sq
    sigma_B_sq = np.zeros(256)
    
    # Step 2: 遍歷所有可能的閾值
    for T in range(256):
        # Class 0: 灰階 <= T
        omega_0 = p[:T+1].sum()
        # Class 1: 灰階 > T
        omega_1 = p[T+1:].sum()
        
        # 避免除以零
        if omega_0 == 0 or omega_1 == 0:
            sigma_B_sq[T] = 0
            continue
        
        # 計算類別平均
        mu_0 = np.sum(k[:T+1] * p[:T+1]) / omega_0
        mu_1 = np.sum(k[T+1:] * p[T+1:]) / omega_1
        
        # 計算類間變異數
        sigma_B_sq[T] = omega_0 * omega_1 * (mu_0 - mu_1)**2
    
    # Step 3: 找最大值
    best_threshold = np.argmax(sigma_B_sq)
    
    return best_threshold, sigma_B_sq

# 測試
threshold, sigma_curve = otsu_threshold_naive(bimodal_img)
print(f"Bimodal 影像的最佳閾值：{threshold}")
print(f"最大類間變異數：{sigma_curve[threshold]:.2f}")

In [ ]:
# 視覺化 sigma_B^2 曲線
def visualize_otsu(image, title):
    """
    視覺化 Otsu 閾值選擇過程。
    """
    threshold, sigma_curve = otsu_threshold_naive(image)
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # 1. 原始影像
    axes[0, 0].imshow(image, cmap='gray', vmin=0, vmax=255)
    axes[0, 0].set_title(f'{title} Original')
    axes[0, 0].axis('off')
    
    # 2. 直方圖 + 閾值線
    hist = compute_histogram(image)
    axes[0, 1].bar(range(256), hist, width=1, color='gray', edgecolor='none')
    axes[0, 1].axvline(threshold, color='red', linestyle='--', linewidth=2, 
                       label=f'Threshold = {threshold}')
    axes[0, 1].set_xlim(0, 255)
    axes[0, 1].set_title('Histogram with Otsu Threshold')
    axes[0, 1].legend()
    
    # 3. sigma_B^2 曲線
    axes[1, 0].plot(range(256), sigma_curve, 'b-', linewidth=1)
    axes[1, 0].axvline(threshold, color='red', linestyle='--', linewidth=2)
    axes[1, 0].scatter([threshold], [sigma_curve[threshold]], color='red', s=100, zorder=5)
    axes[1, 0].set_xlim(0, 255)
    axes[1, 0].set_xlabel('Threshold T')
    axes[1, 0].set_ylabel('Between-Class Variance σ²_B')
    axes[1, 0].set_title(f'σ²_B(T) - Maximum at T={threshold}')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 二值化結果
    binary = (image > threshold).astype(np.uint8) * 255
    axes[1, 1].imshow(binary, cmap='gray', vmin=0, vmax=255)
    axes[1, 1].set_title(f'Binary Result (T={threshold})')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return threshold

# 測試三個影像
t1 = visualize_otsu(bimodal_img, "Bimodal")

In [ ]:
t2 = visualize_otsu(gradient_img, "Gradient")

In [ ]:
t3 = visualize_otsu(text_img, "Text-like")

---

# Part 4: 優化實作

上面的 naive 實作每次都重新計算 omega 和 mu，效率不高。

## 使用累積和 (Cumulative Sum) 優化

定義累積量：
- $\omega_0(T) = \sum_{k=0}^{T} p(k)$（累積機率）
- $\mu_0(T) \cdot \omega_0(T) = \sum_{k=0}^{T} k \cdot p(k)$（累積平均 × 權重）

這些可以用遞推計算：
- $\omega_0(T) = \omega_0(T-1) + p(T)$
- $\mu_0(T) \cdot \omega_0(T) = \mu_0(T-1) \cdot \omega_0(T-1) + T \cdot p(T)$

In [ ]:
def otsu_threshold(image):
    """
    優化版 Otsu 閾值選擇。
    使用累積和避免重複計算。
    
    Parameters
    ----------
    image : np.ndarray
        灰階影像 (uint8)
    
    Returns
    -------
    threshold : int
        最佳閾值 (0-255)
    """
    # 計算直方圖
    hist = compute_histogram(image, bins=256)
    total_pixels = image.size
    p = hist.astype(np.float64) / total_pixels
    
    # 預先計算累積量
    k = np.arange(256)
    
    # 累積機率（omega_0 對於每個 T）
    cumsum_p = np.cumsum(p)  # cumsum_p[T] = omega_0(T)
    
    # 累積平均 × 權重（mu_0 * omega_0 對於每個 T）
    cumsum_kp = np.cumsum(k * p)  # cumsum_kp[T] = sum(k * p(k)) for k <= T
    
    # 全域平均
    mu_total = cumsum_kp[-1]
    
    # 計算 sigma_B^2 for all T
    omega_0 = cumsum_p
    omega_1 = 1 - omega_0
    
    # 避免除以零
    omega_0 = np.clip(omega_0, 1e-10, 1 - 1e-10)
    omega_1 = np.clip(omega_1, 1e-10, 1 - 1e-10)
    
    # mu_0 = cumsum_kp / omega_0
    # mu_1 = (mu_total - cumsum_kp) / omega_1
    mu_0 = cumsum_kp / omega_0
    mu_1 = (mu_total - cumsum_kp) / omega_1
    
    # sigma_B^2 = omega_0 * omega_1 * (mu_0 - mu_1)^2
    sigma_B_sq = omega_0 * omega_1 * (mu_0 - mu_1)**2
    
    # 找最大值
    best_threshold = np.argmax(sigma_B_sq)
    
    return best_threshold

# 驗證與 naive 版本結果相同
t_naive, _ = otsu_threshold_naive(bimodal_img)
t_fast = otsu_threshold(bimodal_img)

print(f"Naive 版本：{t_naive}")
print(f"優化版本：{t_fast}")
print(f"結果一致：{t_naive == t_fast}")

In [ ]:
# 效能比較
import time

# 測試用的較大影像
large_image = create_bimodal_image(500)

# Naive 版本
start = time.time()
for _ in range(10):
    _ = otsu_threshold_naive(large_image)
time_naive = (time.time() - start) / 10

# 優化版本
start = time.time()
for _ in range(10):
    _ = otsu_threshold(large_image)
time_fast = (time.time() - start) / 10

print(f"Naive 版本：{time_naive*1000:.2f} ms")
print(f"優化版本：{time_fast*1000:.2f} ms")
print(f"加速比：{time_naive/time_fast:.1f}x")

---

# Part 5: 為什麼 Otsu 有效？

## 總變異數分解

影像的總變異數 $\sigma_T^2$ 可以分解為：

$$
\sigma_T^2 = \sigma_W^2 + \sigma_B^2
$$

其中：
- $\sigma_W^2$ = 類內變異數 (Within-Class Variance)
- $\sigma_B^2$ = 類間變異數 (Between-Class Variance)

### 類內變異數

$$
\sigma_W^2 = \omega_0 \sigma_0^2 + \omega_1 \sigma_1^2
$$

其中 $\sigma_0^2$ 和 $\sigma_1^2$ 是各類別的內部變異數。

### 關鍵洞見

因為 $\sigma_T^2$ 是固定的（只與影像有關，與閾值無關）：

$$
\max \sigma_B^2 \Leftrightarrow \min \sigma_W^2
$$

**最大化類間變異 = 最小化類內變異**

In [ ]:
def verify_variance_decomposition(image):
    """
    驗證 sigma_T^2 = sigma_W^2 + sigma_B^2。
    """
    hist = compute_histogram(image)
    total = image.size
    p = hist.astype(np.float64) / total
    k = np.arange(256)
    
    # 計算全域變異數
    mu_total = np.sum(k * p)
    sigma_T_sq = np.sum((k - mu_total)**2 * p)
    
    # 取最佳閾值
    T = otsu_threshold(image)
    
    # 計算類間變異數
    omega_0 = p[:T+1].sum()
    omega_1 = p[T+1:].sum()
    
    if omega_0 > 0:
        mu_0 = np.sum(k[:T+1] * p[:T+1]) / omega_0
        sigma_0_sq = np.sum((k[:T+1] - mu_0)**2 * p[:T+1]) / omega_0
    else:
        mu_0, sigma_0_sq = 0, 0
    
    if omega_1 > 0:
        mu_1 = np.sum(k[T+1:] * p[T+1:]) / omega_1
        sigma_1_sq = np.sum((k[T+1:] - mu_1)**2 * p[T+1:]) / omega_1
    else:
        mu_1, sigma_1_sq = 0, 0
    
    sigma_B_sq = omega_0 * omega_1 * (mu_0 - mu_1)**2
    sigma_W_sq = omega_0 * sigma_0_sq + omega_1 * sigma_1_sq
    
    print(f"閾值 T = {T}")
    print(f"σ_T² (總變異)  = {sigma_T_sq:.2f}")
    print(f"σ_B² (類間變異) = {sigma_B_sq:.2f}")
    print(f"σ_W² (類內變異) = {sigma_W_sq:.2f}")
    print(f"σ_B² + σ_W² = {sigma_B_sq + sigma_W_sq:.2f}")
    print(f"驗證：σ_T² ≈ σ_B² + σ_W²: {np.isclose(sigma_T_sq, sigma_B_sq + sigma_W_sq)}")

verify_variance_decomposition(bimodal_img)

---

# Part 6: 練習題

## 練習 1: 實作多閾值 Otsu（三類別）

In [ ]:
def otsu_multi_threshold(image, n_thresholds=2):
    """
    多閾值 Otsu（簡化版，只支援 2 個閾值 = 3 類別）。
    
    Parameters
    ----------
    image : np.ndarray
        灰階影像
    n_thresholds : int
        閾值數量（目前只支援 2）
    
    Returns
    -------
    thresholds : tuple
        最佳閾值組合
    
    注意：這是暴力搜尋，複雜度 O(L^2)，較慢
    """
    if n_thresholds != 2:
        raise ValueError("Only n_thresholds=2 is supported")
    
    hist = compute_histogram(image)
    total = image.size
    p = hist.astype(np.float64) / total
    k = np.arange(256)
    
    mu_total = np.sum(k * p)
    
    best_sigma = 0
    best_thresholds = (0, 0)
    
    # 暴力搜尋所有 (T1, T2) 組合
    for T1 in range(1, 254):
        for T2 in range(T1 + 1, 255):
            # Class 0: k <= T1
            omega_0 = p[:T1+1].sum()
            # Class 1: T1 < k <= T2
            omega_1 = p[T1+1:T2+1].sum()
            # Class 2: k > T2
            omega_2 = p[T2+1:].sum()
            
            if omega_0 == 0 or omega_1 == 0 or omega_2 == 0:
                continue
            
            mu_0 = np.sum(k[:T1+1] * p[:T1+1]) / omega_0
            mu_1 = np.sum(k[T1+1:T2+1] * p[T1+1:T2+1]) / omega_1
            mu_2 = np.sum(k[T2+1:] * p[T2+1:]) / omega_2
            
            # 類間變異數
            sigma_B = (omega_0 * (mu_0 - mu_total)**2 + 
                       omega_1 * (mu_1 - mu_total)**2 + 
                       omega_2 * (mu_2 - mu_total)**2)
            
            if sigma_B > best_sigma:
                best_sigma = sigma_B
                best_thresholds = (T1, T2)
    
    return best_thresholds

# 建立三類別的測試影像
def create_trimodal_image(size=200):
    """三類別影像"""
    np.random.seed(42)
    image = np.zeros((size, size), dtype=np.float64)
    
    # Class 0: 暗（50）
    image[:, :size//3] = 50 + np.random.randn(size, size//3) * 10
    # Class 1: 中（130）
    image[:, size//3:2*size//3] = 130 + np.random.randn(size, size//3) * 10
    # Class 2: 亮（210）
    image[:, 2*size//3:] = 210 + np.random.randn(size, size - 2*size//3) * 10
    
    return np.clip(image, 0, 255).astype(np.uint8)

trimodal = create_trimodal_image(100)

# 找兩個閾值
T1, T2 = otsu_multi_threshold(trimodal, n_thresholds=2)
print(f"最佳閾值：T1={T1}, T2={T2}")

# 視覺化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(trimodal, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('Trimodal Image')
axes[0].axis('off')

hist = compute_histogram(trimodal)
axes[1].bar(range(256), hist, width=1, color='gray')
axes[1].axvline(T1, color='red', linestyle='--', label=f'T1={T1}')
axes[1].axvline(T2, color='blue', linestyle='--', label=f'T2={T2}')
axes[1].set_xlim(0, 255)
axes[1].set_title('Histogram with Thresholds')
axes[1].legend()

# 三類別分割結果
result = np.zeros_like(trimodal)
result[trimodal <= T1] = 0
result[(trimodal > T1) & (trimodal <= T2)] = 127
result[trimodal > T2] = 255

axes[2].imshow(result, cmap='gray', vmin=0, vmax=255)
axes[2].set_title('3-Level Segmentation')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 練習 2: 完整的二值化工具

In [ ]:
def apply_threshold(image, threshold, invert=False):
    """
    對影像套用二值化閾值。
    
    Parameters
    ----------
    image : np.ndarray
        灰階影像
    threshold : int
        閾值
    invert : bool
        是否反轉結果
    
    Returns
    -------
    binary : np.ndarray
        二值化影像 (0 或 255)
    """
    if invert:
        binary = (image <= threshold).astype(np.uint8) * 255
    else:
        binary = (image > threshold).astype(np.uint8) * 255
    
    return binary


def adaptive_threshold(image, block_size=11, C=2):
    """
    自適應閾值（區域 Otsu）。
    
    對影像的每個區塊分別計算 Otsu 閾值。
    適用於光照不均的影像。
    
    Parameters
    ----------
    image : np.ndarray
        灰階影像
    block_size : int
        區塊大小（必須是奇數）
    C : int
        從閾值減去的常數（用於微調）
    
    Returns
    -------
    binary : np.ndarray
        二值化影像
    """
    H, W = image.shape
    half = block_size // 2
    
    # Pad 影像
    padded = np.pad(image, half, mode='reflect')
    
    binary = np.zeros_like(image, dtype=np.uint8)
    
    for i in range(H):
        for j in range(W):
            # 取出區塊
            block = padded[i:i+block_size, j:j+block_size]
            
            # 計算區塊的平均值作為閾值（簡化版）
            local_thresh = block.mean() - C
            
            # 二值化
            if image[i, j] > local_thresh:
                binary[i, j] = 255
    
    return binary


class ThresholdMethods:
    """
    閾值方法集合。
    """
    
    @staticmethod
    def otsu(image):
        """Otsu 自動閾值"""
        return otsu_threshold(image)
    
    @staticmethod
    def binary(image, threshold, invert=False):
        """固定閾值二值化"""
        return apply_threshold(image, threshold, invert)
    
    @staticmethod
    def otsu_binary(image, invert=False):
        """Otsu 自動閾值 + 二值化"""
        T = otsu_threshold(image)
        return apply_threshold(image, T, invert)
    
    @staticmethod
    def adaptive(image, block_size=11, C=2):
        """自適應閾值"""
        return adaptive_threshold(image, block_size, C)

# 測試
tm = ThresholdMethods

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(text_img, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title('Original')

axes[0, 1].imshow(tm.binary(text_img, 128), cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title('Fixed Threshold (T=128)')

otsu_result = tm.otsu_binary(text_img)
axes[0, 2].imshow(otsu_result, cmap='gray', vmin=0, vmax=255)
axes[0, 2].set_title(f'Otsu (T={tm.otsu(text_img)})')

axes[1, 0].imshow(tm.adaptive(text_img, 21, 5), cmap='gray', vmin=0, vmax=255)
axes[1, 0].set_title('Adaptive (block=21)')

axes[1, 1].imshow(tm.otsu_binary(text_img, invert=True), cmap='gray', vmin=0, vmax=255)
axes[1, 1].set_title('Otsu Inverted')

axes[1, 2].axis('off')

for ax in axes.flat:
    if ax.has_data():
        ax.axis('off')

plt.tight_layout()
plt.show()

---

# 總結

## Otsu's Method 核心概念

| 項目 | 說明 |
|------|------|
| 目標 | 自動找到最佳二值化閾值 |
| 原理 | 最大化類間變異數 = 最小化類內變異數 |
| 公式 | $\sigma_B^2 = \omega_0 \omega_1 (\mu_0 - \mu_1)^2$ |
| 優點 | 全自動、計算簡單、效果好 |
| 限制 | 假設雙峰分布，光照不均時效果差 |

## 關鍵函數

- `otsu_threshold(image)`: 計算最佳閾值
- `apply_threshold(image, T)`: 套用閾值二值化
- `otsu_multi_threshold(image, n)`: 多閾值 Otsu

## 下一步

在 02_canny.ipynb 中，我們將實作完整的 **Canny 邊緣偵測**，它結合了：
- Gaussian Smoothing
- Gradient Computation
- Non-Maximum Suppression
- Hysteresis Thresholding

In [ ]:
# 最終驗證
print("=== 函數驗證 ===")

test_funcs = [
    ("compute_histogram(image)", lambda: compute_histogram(bimodal_img)),
    ("otsu_threshold_naive(image)", lambda: otsu_threshold_naive(bimodal_img)),
    ("otsu_threshold(image)", lambda: otsu_threshold(bimodal_img)),
    ("apply_threshold(image, T)", lambda: apply_threshold(bimodal_img, 100)),
    ("otsu_multi_threshold(image, 2)", lambda: otsu_multi_threshold(trimodal, 2)),
    ("adaptive_threshold(image)", lambda: adaptive_threshold(text_img[:50, :50])),
    ("ThresholdMethods.otsu(image)", lambda: ThresholdMethods.otsu(bimodal_img)),
    ("ThresholdMethods.otsu_binary(image)", lambda: ThresholdMethods.otsu_binary(bimodal_img)),
]

for name, func in test_funcs:
    try:
        result = func()
        print(f"✓ {name}")
    except Exception as e:
        print(f"✗ {name}: {e}")

print("\n所有函數驗證完成！")